In [1]:
%load_ext autoreload
%autoreload 2
from boxes import *
from learner import *
import math
import matplotlib.pyplot as plt
import os
import wandb
import pickle

%matplotlib inline

In [2]:
def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed) # cpu
    torch.cuda.manual_seed_all(seed)  # gpu
    
set_seed(42)

torch.set_printoptions(precision=16)

In [3]:
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
use_cuda

False

In [273]:
PATH = 'data/ontologies/anatomy/'

# aligment training split
ats = 0.8

# Transitive closure
Transitive_Closure = False

if Transitive_Closure:
    tc = "tc_"
else:
    tc = ""

# Data in unary.tsv are probabilites separated by newlines. The probability on line n is P(n), where n is the id assigned to the nth element.
unary_prob = torch.from_numpy(np.loadtxt(f'{PATH}unary/unary.tsv')).float().to(device)
num_boxes = unary_prob.shape[0]

# We're going to use random negative sampling during training, so no need to include negatives in our training data itself
train = Probs.load_from_julia(PATH, f'tr_pos_{tc}{ats}.tsv', f'tr_neg_{ats}.tsv', ratio_neg = 0).to(device)

# The dev set will have a fixed set of negatives, however.
dev = Probs.load_from_julia(PATH, f'dev_align_pos_{ats}.tsv', f'dev_align_neg_{ats}.tsv', ratio_neg = 1).to(device)

# This set is used just for evaluation purposes after training
tr_align = Probs.load_from_julia(PATH, f'tr_align_pos_{ats}.tsv', f'tr_align_neg_{ats}.tsv', ratio_neg = 1).to(device)


In [274]:
with open('data/ontologies/anatomy/human.pickle', 'rb') as f:
    human_pickle = pickle.load(f)
    
with open('data/ontologies/anatomy/mouse.pickle', 'rb') as f:
    mouse_pickle = pickle.load(f)
    
with open('data/ontologies/anatomy/entities.pickle', 'rb') as f:
    entity_pickle = pickle.load(f)

In [275]:
mouse_eval = Probs.load_from_julia(PATH, 'human_dev_pos.tsv', 'human_dev_neg.tsv', ratio_neg = 1).to(device)
human_eval = Probs.load_from_julia(PATH, 'mouse_dev_pos.tsv', 'mouse_dev_neg.tsv', ratio_neg = 1).to(device)

In [276]:
import torch
from torch import Tensor # for type annotations
import torch.nn.functional as F
from typing import *


def intersection(A: Tensor, B: Tensor) -> Tensor:
    """
    :param A: Tensor(..., zZ, dim)
    :param B: Tensor(..., zZ, dim)
    :return: Tensor(..., zZ, dim), box embeddings for A intersect B
    """
    z = torch.max(A[...,0,:], B[...,0,:])
    Z = torch.min(A[...,1,:], B[...,1,:])
    return torch.stack((z, Z), dim=-2)

def neg_edge_adjustment(A: Tensor) -> Tensor:
    """
    :param A: Tensor(..., zZ, dim)
    
    Replace "negative" edges with their mean.
    
    (TODO: optimize this)
    """
    center_of_meet = torch.mean(A, dim=-2)
    neg_edges_mask = ((A[...,1,:] - A[...,0,:]) >= 0)
    neg_edges_mask_stack = torch.stack((neg_edges_mask, neg_edges_mask), dim=-2)
    center_of_meet_stack = torch.stack((center_of_meet, center_of_meet), dim=-2)
    return torch.where(neg_edges_mask_stack, A, center_of_meet_stack)

def join(A: Tensor, B: Tensor) -> Tensor:
    """
    :param A: Tensor(model, pair, zZ, dim)
    :param B: Tensor(model, pair, zZ, dim)
    :return: Tensor(model, pair, zZ, dim), box embeddings for the smallest box which contains A and B
    """
    z = torch.min(A[:,:,0], B[:,:,0])
    Z = torch.max(A[:,:,1], B[:,:,1])
    return torch.stack((z, Z), dim=2)


def clamp_volume(boxes: Tensor) -> Tensor:
    """
    :param boxes: Tensor(... zZ, dim)
    :return: Tensor(...) of volumes
    """
    return torch.prod((boxes[...,1,:] - boxes[...,0,:]).clamp_min(0), dim=-1)


def soft_volume(boxes: Tensor) -> Tensor:
    """
    :param sidelengths: Tensor(model, box, dim)
    :return: Tensor(model, box) of volumes
    """
    return torch.prod(F.softplus(boxes[:,:,1] - boxes[:,:,0]), dim=-1)


def log_clamp_volume(boxes: Tensor, eps:float = torch.finfo(torch.float32).tiny) -> Tensor:
    """
    :param boxes: Tensor(model, box, zZ, dim)
    :return: Tensor(model, box) of volumes
    """
    return torch.sum(torch.log((boxes[:,:,1] - boxes[:,:,0]).clamp_min(0) + eps), dim=-1)


def log_soft_volume(boxes: Tensor, eps:float = torch.finfo(torch.float32).tiny) -> Tensor:
    """
    :param sidelengths: Tensor(model, box, dim)
    :return: Tensor(model, box) of volumes
    """
    return torch.sum(torch.log(F.softplus(boxes[:,:,1] - boxes[:,:,0]) + eps), dim=-1)


def smallest_containing_box(boxes: Tensor) -> Tensor:
    """
    Returns the smallest box which contains all boxes in `boxes`.
    
    :param boxes: Box embedding of shape (model, box, zZ, dim)
    :return: Tensor of shape (model, 1, zZ, dim)
    """
    z = boxes[:,:,0]
    Z = boxes[:,:,1]
    min_z, _ = torch.min(z, dim=1, keepdim=True)
    max_Z, _ = torch.max(Z, dim=1, keepdim=True)
    return torch.stack((min_z, max_Z), dim=2)

def smallest_containing_box_outside_unit_cube(boxes: Tensor) -> Tensor:
    """
    Returns the smallest box which contains all boxes in `boxes` and the unit cube.

    :param boxes: Box embedding of shape (model, box, zZ, dim)
    :return: Tensor of shape (model, 1, zZ, dim)
    """
    z = boxes[:,:,0]
    Z = boxes[:,:,1]
    min_z, _ = torch.min(z, dim=1, keepdim=True)
    max_Z, _ = torch.max(Z, dim=1, keepdim=True)
    min_z = min_z.clamp_max(0)
    max_Z = max_Z.clamp_min(1)
    return torch.stack((min_z, max_Z), dim=2)


def detect_small_boxes(boxes: Tensor, vol_func: Callable = clamp_volume, min_vol: float = 1e-20) -> Tensor:
    """
    Returns the indices of boxes with volume smaller than eps.

    :param boxes: box parametrization as Tensor(model, box, z/Z, dim)
    :param vol_func: function taking in side lengths and returning volumes
    :param min_vol: minimum volume of boxes
    :return: masked tensor which selects boxes whose side lengths are less than min_vol
    """
    return vol_func(boxes) < min_vol


def replace_Z_by_cube(boxes: Tensor, indices: Tensor, cube_vol: float = 1e-20) -> Tensor:
    """
    Returns a new Z parameter for boxes for which those boxes[indices] are now replaced by cubes of size cube_vol

    :param boxes: box parametrization as Tensor(model, box, z/Z, dim)
    :param indices: box indices to replace by a cube
    :param cube_vol: volume of cube
    :return: tensor representing the Z parameter
    """
    return boxes[:, :, 0][indices] + cube_vol ** (1 / boxes.shape[-1])



def replace_Z_by_cube_(boxes: Tensor, indices: Tensor, cube_vol: float = 1e-20) -> Tensor:
    """
    Replaces the boxes indexed by `indices` by a cube of volume `min_vol` with the same z coordinate

    :param boxes: box parametrization as Tensor(model, box, z/Z, dim)
    :param indices: box indices to replace by a cube
    :param cube_vol: volume of cube
    :return: tensor representing the box parametrization with those boxes
    """
    boxes[:, :, 1][indices] = replace_Z_by_cube(boxes, indices, cube_vol)


def disjoint_boxes_mask(A: Tensor, B: Tensor) -> Tensor:
    """
    Returns a mask for when A and B are disjoint.
    Note: This is symmetric with respect to the arguments.
    """
    return ((B[:,:,1] <= A[:,:,0]) | (B[:,:,0] >= A[:,:,1])).any(dim=-1)


def overlapping_boxes_mask(A: Tensor, B: Tensor) -> Tensor:
    return disjoint_boxes_mask(A, B) ^ 1


def containing_boxes_mask(A: Tensor, B: Tensor) -> Tensor:
    """
    Returns a mask for when B contains A.
    Note: This is *not* symmetric with respect to it's arguments!
    """
    return ((B[:,:,1] >= A[:,:,1]) & (B[:,:,0] <= A[:,:,0])).all(dim=-1)


def needing_pull_mask(A: Tensor, B: Tensor, target_prob_B_given_A: Tensor) -> Tensor:
    return (target_prob_B_given_A != 0) & disjoint_boxes_mask(A, B)


def needing_push_mask(A: Tensor, B: Tensor, target_prob_B_given_A: Tensor) -> Tensor:
    return (target_prob_B_given_A != 1) & containing_boxes_mask(A, B)


In [277]:
import torch
from torch import Tensor
import scipy.stats as spstats # For Spearman r
from sklearn.metrics import roc_curve, precision_recall_curve  # for roc_curve


def metric_hard_accuracy(model, data_in, data_out):
    hard_pred = model(data_in, is_align=torch.tensor(0))["P(A|B)"] > 0.5
    return (data_out == hard_pred.float()).float().mean()


def metric_hard_f1(model, data_in, data_out):
    hard_pred = model(data_in, is_align=torch.tensor(0))["P(A|B)"] > 0.5
    true_pos = data_out[hard_pred==1].sum()
    total_pred_pos = (hard_pred==1).sum().float()
    total_actual_pos = data_out.sum().float()
    precision = true_pos / total_pred_pos
    recall = true_pos / total_actual_pos
    return 2 * (precision*recall) / (precision + recall)

def metric_hard_accuracy_align(model, data_in, data_out, threshold:float):
    A_given_B = data_in[::2]
    B_given_A = data_in[1::2,:]
    data_out = data_out[::2]

    align_probs = torch.stack((model(A_given_B, is_align=torch.tensor(1))["P(A|B)"], model(B_given_A, is_align=torch.tensor(1))["P(A|B)"]), dim=1)
    p = torch.min(align_probs, dim=1).values
    hard_pred = p > threshold

    return (data_out == hard_pred).float().mean()

def metric_hard_f1_align(model, data_in, data_out, threshold:float):
    A_given_B = data_in[::2]
    B_given_A = data_in[1::2,:]
    data_out = data_out[::2]

    align_probs = torch.stack((model(A_given_B, is_align=torch.tensor(1))["P(A|B)"], model(B_given_A, is_align=torch.tensor(1))["P(A|B)"]), dim=1)
    p = torch.min(align_probs, dim=1).values
    hard_pred = p > threshold

    true_pos = data_out[hard_pred==1].sum()
    total_pred_pos = (hard_pred==1).sum().float()
    total_actual_pos = data_out.sum().float()

    precision = true_pos / total_pred_pos
    recall = true_pos / total_actual_pos

    return 2 * (precision*recall) / (precision + recall)

def metric_hard_accuracy_align_mean(model, data_in, data_out, threshold):
    A_given_B = data_in[::2]
    B_given_A = data_in[1::2,:]
    data_out = data_out[::2]

    align_probs = torch.stack((model(A_given_B, is_align=torch.tensor(1))["P(A|B)"], model(B_given_A, is_align=torch.tensor(1))["P(A|B)"]), dim=1)
    p = torch.mean(align_probs, dim=1)
    hard_pred = p > threshold

    return (data_out == hard_pred).float().mean()

def metric_hard_f1_align_mean(model, data_in, data_out, threshold):
    A_given_B = data_in[::2]
    B_given_A = data_in[1::2,:]
    data_out = data_out[::2]

    align_probs = torch.stack((model(A_given_B, is_align=torch.tensor(1))["P(A|B)"], model(B_given_A, is_align=torch.tensor(1))["P(A|B)"]), dim=1)
    p = torch.mean(align_probs, dim=1)
    hard_pred = p > threshold

    true_pos = data_out[hard_pred==1].sum()
    total_pred_pos = (hard_pred==1).sum().float()
    total_actual_pos = data_out.sum().float()

    precision = true_pos / total_pred_pos
    recall = true_pos / total_actual_pos

    return 2 * (precision*recall) / (precision + recall)


In [278]:
import torch
from torch.utils.data import Dataset
from typing import *
from dataclasses import dataclass, field
import wandb

try:
    from IPython import get_ipython
    if 'IPKernelApp' not in get_ipython().config:  # pragma: no cover
        raise ImportError("console")
except:
    pass
else:
    import ipywidgets as widgets
    from IPython.core.display import HTML, display

if TYPE_CHECKING:
    from learner import Learner, Recorder


class Callback:
    def learner_post_init(self, learner: Learner):
        pass

    def train_begin(self, learner: Learner):
        pass

    def epoch_begin(self, learner: Learner):
        pass

    def batch_begin(self, learner: Learner):
        pass

    def backward_end(self, learner: Learner):
        pass

    def batch_end(self, learner: Learner):
        pass

    def epoch_end(self, learner: Learner):
        pass

    def train_end(self, learner: Learner):
        pass

    def eval_align(self, learner: Learner, threshold:float):
        pass

    def metric_plots(self, l: Learner):
        pass

    def eval_end(self, l: Learner):
        pass

    def bias_metric(self, l: Learner):
        pass

class CallbackCollection:

    def __init__(self, *callbacks: Callback):
        self._callbacks = callbacks

    def __call__(self, action: str, *args, **kwargs):
        for c in self._callbacks:
            getattr(c, action)(*args, **kwargs)

    def __getattr__(self, action: str):
        return lambda *args, **kwargs: self.__call__(action, *args, **kwargs)


@dataclass
class GradientClipping(Callback):
    min: float = None
    max: float = None

    def backward_end(self, learner: Learner):
        for param in learner.model.parameters():
            if param.grad is not None:
                param.grad = param.grad.clamp(self.min, self.max)


@dataclass
class LossCallback(Callback):
    recorder: Recorder
    ds: Dataset
    weighted: bool = True

    @torch.no_grad()
    def train_begin(self, learner: Learner):
        self.epoch_end(learner)

    @torch.no_grad()
    def epoch_end(self, l: Learner):
        data_in, data_out = self.ds[:]
        if l.categories:
            split_in, split_out = l.split_data(data_in, data_out, split=2737)
            
            model_pred = []
            count = 0
            for item in split_in:
                if len(item)>0:
                    if count<2:
                        model_pred.append(l.model(item, is_align=torch.tensor(0)))
                    else:
                        model_pred.append(l.model(item, is_align=torch.tensor(1)))
                else:
                    model_pred.append({'P(A|B)':l.TensorNaN(device=data_in.device)})
                count+=1

            #model_pred = [l.model(item) if len(item)>0 else {'P(A|B)':l.TensorNaN(device=data_in.device)} for item in split_in]
            l.loss_fn(model_pred, split_out, l, self.recorder, weighted=self.weighted, categories=True)  
        else:
            output = l.model(data_in, is_align=torch.tensor(0))
            l.loss_fn(output, data_out, l, self.recorder, weighted=self.weighted) # this logs the data to the recorder


@dataclass
class MetricCallback(Callback):
    recorder: Recorder
    ds: Dataset
    data_categories: str
    metric: Callable
    use_wandb: bool = False
    name: Union[str, None] = None

    def __post_init__(self):
        if self.name is None:
            self.name = self.metric.__name__
        self.name = self.recorder.get_unique_name(self.name)

    @torch.no_grad()
    def train_begin(self, learner: Learner):
        self.epoch_end(learner)

    @torch.no_grad()
    def epoch_end(self, l: Learner):
        data_in, data_out = self.ds[:]
        metric_val = self.metric(l.model, data_in, data_out)
        self.recorder.update_({self.name: metric_val}, l.progress.current_epoch_iter)
        
        print("evaluation_" + self.data_categories + "_" + self.name, str(metric_val))
        
        if self.use_wandb:
            metric_name = "evaluation_" + self.data_categories + "_" + self.name
            wandb.log({metric_name: metric_val})

@dataclass
class EvalAlignment(Callback):
    recorder: Recorder
    ds: Dataset
    data_categories: str
    metric: callable
    use_wandb: bool = False
    name: Union[str, None] = None
        

    def __post_init__(self):
        if self.name is None:
            self.name = self.metric.__name__
        self.name = self.recorder.get_unique_name(self.name)

    @torch.no_grad()
    def eval_align(self, l: Learner, threshold: float):
        data_in, data_out = self.ds[:]
        metric_val = self.metric(l.model, data_in, data_out, threshold)
        self.recorder.update_({self.name: metric_val}, threshold)
        
        print("align_evaluation_" + self.data_categories + "_" + str(threshold) + "_" + self.name, str(metric_val))
        
        if self.use_wandb:
            metric_name = "align_evaluation_" + self.data_categories + "_" + str(threshold) + "_" + self.name
            wandb.log({metric_name: metric_val})


In [286]:
import torch
from torch import Tensor
import torch.nn as nn
from torch.nn import Module, Parameter
import torch.nn.functional as F


################################################
# Box Parametrization Layers
################################################
default_init_min_vol = torch.finfo(torch.float32).tiny



class BoxParam(Module):
    """
    An example class for creating a box parametrization.
    Don't inherit from this, it is just an example which contains the methods for a class to be used as a BoxParam
    layer. Refer to the docstring of the functions when implementing your own BoxParam.

    Note: to avoid naming conflicts with min/max functions, we refer to the min coordinate for a box as `z`, and the
    max coordinate as `Z`.
    """

    def __init__(self, num_models:int, num_boxes:int, dim:int, **kwargs):
        """
        Creates the Parameters used for the representation of boxes.

        :param num_models: Number of models
        :param num_boxes: Number of boxes
        :param dim: Dimension
        :param kwargs: Unused for now, but include this for future possible parameters.
        """
        # Remember to call:
        super().__init__()
        raise NotImplemented


    def forward(self, box_indices = slice(None, None, None), **kwargs) -> Tensor:
        """
        Returns a Tensor representing the boxes specified by `box_indices` in the form they should be used for training.

        :param box_indices: Slice, List, or Tensor of the box indices
        :param kwargs: Unused for now, but include this for future possible parameters.
        :return: Tensor of shape (model, id, zZ, dim).
        """
        raise NotImplemented


class Boxes(Module):
    """
    Parametrize boxes using the min coordinate and max coordinate,
    initialized to be in the unit hypercube.

    self.boxes[model, box, min/max, dim] \in [0,1]

    In this parametrization, the min and max coordinates are explicitly stored
    in separate dimensions (as shown above), which means that care must be
    taken to preserve max > min while training. (See MinBoxSize Callback.)
    """

    def __init__(self, num_models: int, num_boxes: int, dims: int,
                 init_min_vol: float = default_init_min_vol, method = "gibbs", gibbs_iter: int = 2000, **kwargs):
        super().__init__()
        self.boxes = Parameter(initialize_boxes_in_unit_cube((num_models, num_boxes), dims, init_min_vol, method, gibbs_iter, **kwargs))

    def forward(self, box_indices = slice(None, None, None), **kwargs) -> Tensor:
        """
        Returns a Tensor representing the box embeddings specified by box_indices.

        :param box_indices: Slice, List, or Tensor of the box indices
        :param kwargs: Unused for now, but include this for future possible parameters.
        :return: NamedTensor of shape (model, id, zZ, dim).
        """
        return self.boxes[:, box_indices]



class MinMaxSigmoidBoxes(Module):
    """
    Parametrize boxes using sigmoid to make them always valid and contained within the unit cube.

    self.boxes[model, box, 2, dim] in Reals


    In this parametrization, we first convert to the unit cube:

    unit_cube_boxes = torch.sigmoid(self.boxes)  # shape: (model, box, 2, dim)

    We now select the z/Z coordinates by taking the min/max over axis 2, i.e.

    z, _ = torch.min(unit_cube_boxes, dim=2)
    Z, _ = torch.max(unit_cube_boxes, dim=2)
    """

    def __init__(self, num_models: int, num_boxes: int, dim: int, init_min_vol: float = default_init_min_vol,  **kwargs):
        super().__init__()
        unit_boxes = Boxes(num_models, num_boxes, dim, init_min_vol, **kwargs)
        self._from_UnitBoxes(unit_boxes)
        del unit_boxes


    def _from_UnitBoxes(self, unit_boxes:Boxes):
        boxes = unit_boxes().detach().clone()
        self.boxes = Parameter(torch.log(boxes / (1-boxes)))


    def forward(self, box_indices = slice(None, None, None), **kwargs) -> Tensor:
        """
        Returns a Tensor representing the box embeddings specified by box_indices.

        :param box_indices: A NamedTensor of the box indices
        :param kwargs: Unused for now, but include this for future possible parameters.
        :return: Tensor of shape (model, id, zZ, dim).
        """
        unit_cube_boxes = torch.sigmoid(self.boxes)
        z, _ = torch.min(unit_cube_boxes, dim=2)
        Z, _ = torch.max(unit_cube_boxes, dim=2)
        return torch.stack((z,Z), dim=2)
    
class AlignmentBoxes(Module):
    """
    Parametrize boxes using sigmoid to make them always valid and contained within the unit cube.

    self.boxes[model, box, 2, dim] in Reals


    In this parametrization, we first convert to the unit cube:

    unit_cube_boxes = torch.sigmoid(self.boxes)  # shape: (model, box, 2, dim)

    We now select the z/Z coordinates by taking the min/max over axis 2, i.e.

    z, _ = torch.min(unit_cube_boxes, dim=2)
    Z, _ = torch.max(unit_cube_boxes, dim=2)
    """

    def __init__(self, num_models: int, num_boxes: int, dim: int, init_min_vol: float = default_init_min_vol,  **kwargs):
        super().__init__()
        unit_boxes = Boxes(num_models, num_boxes, dim, init_min_vol, **kwargs)
        self._from_UnitBoxes(unit_boxes)
        self.fc = nn.Linear(dim, dim)
        del unit_boxes


    def _from_UnitBoxes(self, unit_boxes:Boxes):
        boxes = unit_boxes().detach().clone()
        self.boxes = Parameter(torch.log(boxes / (1-boxes)))


    def forward(self, box_indices = slice(None, None, None), is_align=torch.tensor(0), **kwargs) -> Tensor:
        """
        Returns a Tensor representing the box embeddings specified by box_indices.

        :param box_indices: A NamedTensor of the box indices
        :param kwargs: Unused for now, but include this for future possible parameters.
        :return: Tensor of shape (model, id, zZ, dim).
        """
        unit_cube_boxes = torch.sigmoid(self.boxes)
        z, _ = torch.min(unit_cube_boxes, dim=2)
        Z, _ = torch.max(unit_cube_boxes, dim=2)

        if is_align.tolist() == 1:
#             print("z before1:", z[0][10])
#             print("w:", self.fc.weight)
            z = self.fc(z)
            Z = self.fc(Z)

        return torch.stack((z,Z), dim=2)


###############################################
# Downstream Model
###############################################

class WeightedSum(Module):
    def __init__(self, num_models: int) -> None:
        super().__init__()
        self.weights = Parameter(torch.rand(num_models))

    def forward(self, box_vols: Tensor) -> Tensor:
        return (F.softmax(self.weights, dim=0).unsqueeze(0) @ box_vols).squeeze()


class LogWeightedSum(Module):
    def __init__(self, num_models: int) -> None:
        super().__init__()
        self.weights = Parameter(torch.rand(num_models))

    def forward(self, log_box_vols: Tensor) -> Tensor:
        return (torch.logsumexp(self.weights + log_box_vols, 0) - torch.logsumexp(self.weights, 0))


class BoxModel(Module):
    def __init__(self, BoxParamType: type, vol_func: Callable,
                 num_models:int, num_boxes:int, dims:int,
                 init_min_vol: float = default_init_min_vol, universe_box: Optional[Callable] = None, **kwargs):
        super().__init__()
        self.box_embedding = BoxParamType(num_models, num_boxes, dims, init_min_vol, **kwargs)
        self.vol_func = vol_func

        if universe_box is None:
            z = torch.zeros(dims)
            Z = torch.ones(dims)
            self.universe_box = lambda _: torch.stack((z,Z))[None, None]
            self.universe_vol = lambda _: self.vol_func(self.universe_box(None)).squeeze()
            self.clamp = True
        else:
            self.universe_box = universe_box
            self.universe_vol = lambda b: self.vol_func(self.universe_box(b))
            self.clamp = False

        self.weights = WeightedSum(num_models)

    def forward(self, box_indices: Tensor, is_align: Tensor) -> Dict:
        #print("is_align in BoxModel:", is_align)
        # Unary
        box_embeddings_orig = self.box_embedding(is_align = is_align)
        if self.clamp:
            box_embeddings = box_embeddings_orig.clamp(0,1)
        else:
            box_embeddings = box_embeddings_orig

        universe_vol = self.universe_vol(box_embeddings)

        unary_probs = self.weights(self.vol_func(box_embeddings) / universe_vol)

        # Conditional
        A = box_embeddings[:, box_indices[:,0]]
        B = box_embeddings[:, box_indices[:,1]]
        A_int_B_vol = self.weights(self.vol_func(intersection(A, B)) / universe_vol) + torch.finfo(torch.float32).tiny
        B_vol = unary_probs[box_indices[:,1]] + torch.finfo(torch.float32).tiny
        P_A_given_B = torch.exp(torch.log(A_int_B_vol) - torch.log(B_vol))
        
        # symmetric same
        # print("you are in right place!")
#         A = box_embeddings[:, box_indices[:,0]]
#         B = box_embeddings[:, box_indices[:,1]]
#         A_int_B_vol = self.weights(self.vol_func(intersection(A, B)) / universe_vol) + torch.finfo(torch.float32).tiny
#         A_join_B_vol = self.weights(self.vol_func(join(A, B)) / universe_vol) + torch.finfo(torch.float32).tiny
#         P_A_given_B = torch.exp(torch.log(A_int_B_vol) - torch.log(A_join_B_vol))

        return {
            "unary_probs": unary_probs,
            "box_embeddings_orig": box_embeddings_orig,
            "A": A,
            "B": B,
            "P(A|B)": P_A_given_B,
        }


In [287]:
dims = 16
lr = 0.07342406890949607
#lr = 0.2
rns_ratio = 5
#box_type = MinMaxSigmoidBoxes, AlignmentBoxes
use_unary = False
unary_weight = 1e-2

In [288]:
box_model = BoxModel(
    BoxParamType=AlignmentBoxes,
    vol_func=soft_volume,
    num_models=1,
    num_boxes=num_boxes,
    dims=dims,
    method="tree").to(device)


#### IF YOU ARE LOADING FROM JULIA WITH ratio_neg=0, train_dl WILL ONLY CONTAIN POSITIVE EXAMPLES
#### THIS MEANS YOUR MODEL SHOULD USE NEGATIVE SAMPLING DURING TRAINING
train_dl = TensorDataLoader(train, batch_size=2**6, shuffle=True)

mouse_dl = TensorDataLoader(mouse_eval, batch_size=2**6)
human_dl = TensorDataLoader(human_eval, batch_size=2**6)

eval_dl = [mouse_dl, human_dl]

opt = torch.optim.Adam(box_model.parameters(), lr=lr)

In [289]:
def mean_cond_kl_loss(model_out: ModelOutput, target: Tensor, eps: float = torch.finfo(torch.float32).tiny) -> Tensor:
    return kl_div_sym(model_out["P(A|B)"], target, eps).mean()

def human_cond_kl_loss(model_out: ModelOutput, target: Tensor, eps: float = torch.finfo(torch.float32).tiny) -> Tensor:
    return kl_div_sym(model_out["P(A|B)"], target, eps).mean()

def mouse_cond_kl_loss(model_out: ModelOutput, target: Tensor, eps: float = torch.finfo(torch.float32).tiny) -> Tensor:
    return kl_div_sym(model_out["P(A|B)"], target, eps).mean()

def align_cond_kl_loss(model_out: ModelOutput, target: Tensor, eps: float = torch.finfo(torch.float32).tiny) -> Tensor:
    return kl_div_sym(model_out["P(A|B)"], target, eps).mean()

# See boxes/loss_functions.py file for more options. Note that you may have to changed them to fit your use case.
# Also note that "kl_div_sym" is just binary cross-entropy.

In [290]:
global my_batch_in
global my_batch_out
my_batch_in = None
my_batch_out = None

@dataclass
class Learner:
    train_dl: DataLoader
    model: Module
    loss_fn: Callable
    opt: optim.Optimizer
    callbacks: CallbackCollection = field(default_factory=CallbackCollection)
    recorder: Recorder = field(default_factory=Recorder)
    categories: bool = False
    use_wandb: bool = False
    reraise_keyboard_interrupt: bool = False
    reraise_stop_training_exceptions: bool = False

    def __post_init__(self):
        self.progress = Progress(0,0,len(self.train_dl))
        self.callbacks.learner_post_init(self)

    #the split parameter will be used to find human/mouse/align data, so you need change it when using diff dataset(index)
    def split_data(self, batch_in, batch_out, split):
        category = torch.zeros(size=(batch_in.shape[0],), dtype=int)

        batch_class = batch_in > split

        for i, (a,b) in enumerate(batch_class):
            if not a and not b:
                category[i] = 0
            elif a and b:
                category[i] = 1
            else:
                category[i] = 2

        self.mouse_in = batch_in[category == 0]
        self.human_in = batch_in[category == 1]
        self.align_in = batch_in[category == 2]

        self.mouse_out = batch_out[category == 0]
        self.human_out = batch_out[category == 1]
        self.align_out = batch_out[category == 2]

        # INPUT TO THE MODEL:
        data_in = (self.mouse_in, self.human_in, self.align_in)
        # TARGET/LABEL:
        data_out = (self.mouse_out, self.human_out, self.align_out)

        return data_in, data_out

    def TensorNaN(self, size:Union[None,List[int], Tuple[int]]=None, device=None, requires_grad:bool=True):
        if size is None:    
            return torch.tensor(float('nan'), device=device, requires_grad=requires_grad)
        else:
            return float('nan') * torch.zeros(size=size, device=device, requires_grad=requires_grad)


    def train(self, epochs, progress_bar = True):
        global my_batch_in
        global my_batch_out
        try:
            self.callbacks.train_begin(self)
            for epoch in trange(epochs, desc="Overall Training:", disable=not progress_bar):
                self.callbacks.epoch_begin(self)
                for iteration, batch in enumerate(tqdm(self.train_dl, desc="Current Batch:", leave=False, disable=not progress_bar)):
                    if len(batch) == 2: # KLUDGE
                        self.batch_in, self.batch_out = batch
                    else:
                        self.batch_in = batch[0]
                        self.batch_out = None
                    self.progress.increment()
                    self.callbacks.batch_begin(self)
                    self.opt.zero_grad()
                    
                    if self.categories:
                        self.data_in, self.data_out = self.split_data(self.batch_in, self.batch_out, split=2737)
                        #2737 is max mouse index  
                        self.model_pred = []
                        count = 0
                        for item in self.data_in:
                            if len(item)>0:
                                if count<2:
                                    self.model_pred.append(self.model(item, is_align=torch.tensor(0)))
                                else:
                                    self.model_pred.append(self.model(item, is_align=torch.tensor(0)))
                            else:
                                self.model_pred.append({'P(A|B)':self.TensorNaN(device=self.batch_in.device)})
                            count+=1
                        #self.model_pred = [self.model(item, is_align=is_align) if len(item)>0 else {'P(A|B)':self.TensorNaN(device=self.batch_in.device)} for item in self.data_in]
                        self.loss = self.loss_fn(self.model_pred, self.data_out, self, self.recorder, categories=True, use_wandb=self.use_wandb)                        
                    else:
                        self.model_out = self.model(self.batch_in, is_align=False)
                        if self.batch_out is None:
                            self.loss = self.loss_fn(self.model_out, self, self.recorder, categories=True, use_wandb=self.use_wandb)
                        else:
                            self.loss = self.loss_fn(self.model_out, self.batch_out, self, self.recorder, categories=True, use_wandb=self.use_wandb)
                        
                    # Log metrics inside your training loop
                    if self.use_wandb:
                        metrics = {'epoch': epoch, 'loss': self.loss}
                        wandb.log(metrics)

                    # print(self.recorder.dataframe)
                    self.loss.backward()
                    self.callbacks.backward_end(self)
                    self.opt.step()
                    self.callbacks.batch_end(self)
                # print(self.recorder.dataframe)
                
                # run evaluating at the end of every epoch
                #self.evaluation(np.arange(0.1, 1, 0.1))
                self.evaluation([0.5])
                
                
                self.callbacks.epoch_end(self)
        except StopTrainingException as e:
            print(e)
            if self.reraise_stop_training_exceptions:
                raise e
        except KeyboardInterrupt:
            print(f"Stopped training at {self.progress.partial_epoch_progress()} epochs due to keyboard interrupt.")
            if self.reraise_keyboard_interrupt:
                raise KeyboardInterrupt
        finally:
            self.callbacks.train_end(self)


    def evaluation(self, trials, progress_bar=True):
        with torch.no_grad():
            # self.callbacks.eval_begin(self)
            for t in trials:
                self.callbacks.eval_align(self, t)
            self.callbacks.metric_plots(self)
            self.callbacks.bias_metric(self)
            self.callbacks.eval_end(self)

In [291]:
# For this dataset we had unary probabilities as well as conditional probabilities. Our loss function will be a sum of these, which is provided by the following loss function wrapper:

# if use_unary:
#     loss_func = LossPieces(mean_cond_kl_loss, (unary_weight, mean_unary_kl_loss(unary_prob)))
# else:
#     loss_func = LossPieces(mean_cond_kl_loss)

loss_func = LossPieces( (0.05, mouse_cond_kl_loss), (0.05 ,human_cond_kl_loss), (0.05, align_cond_kl_loss))

metrics = [metric_hard_accuracy, metric_hard_f1]
align_metrics = [metric_hard_accuracy_align, metric_hard_f1_align, metric_hard_accuracy_align_mean, metric_hard_f1_align_mean]

rec_col = RecorderCollection()

#threshold = np.arange(0.1, 1, 0.1)
threshold = [0.5]

callbacks = CallbackCollection(
    LossCallback(rec_col.train, train),
    LossCallback(rec_col.dev, dev),
#     *(MetricCallback(rec_col.dev, dev, "dev", m) for m in metrics),
#     *(MetricCallback(rec_col.train, train, "train", m) for m in metrics),
    *(MetricCallback(rec_col.onto, human_eval, "human", m) for m in metrics),
    *(MetricCallback(rec_col.onto, mouse_eval, "mouse", m) for m in metrics),
    *(EvalAlignment(rec_col.train_align, tr_align, "train_align", m) for m in align_metrics),
    *(EvalAlignment(rec_col.dev_align, dev, "dev_align", m) for m in align_metrics),
    #JustGiveMeTheData(rec_col.probs, dev, get_probabilities),
    #BiasMetric(rec_col.bias, dev, pct_of_align_cond_on_human_as_min),
    #PlotMetrics(rec_col.dev_roc_plot, dev, roc_plot),
    #PlotMetrics(rec_col.dev_pr_plot, dev, pr_plot),
    #PlotMetrics(rec_col.tr_roc_plot, tr_align, roc_plot),
    #PlotMetrics(rec_col.tr_pr_plot, tr_align, pr_plot),
    #MetricCallback(rec_col.train, train, 'train', metric_pearson_r),
    #MetricCallback(rec_col.train, train, 'train', metric_spearman_r),
    #MetricCallback(rec_col.dev, dev, 'dev', metric_pearson_r),
    #MetricCallback(rec_col.dev, dev, 'dev', metric_spearman_r),
#     MetricCallback(rec_col.train, train, 'train', mean_reciprocal_rank),
#     MetricCallback(rec_col.dev, dev, 'dev', mean_reciprocal_rank),
#     PercentIncreaseEarlyStopping(rec_col.dev, "mean_cond_kl_loss", 0.25, 10),
#     PercentIncreaseEarlyStopping(rec_col.dev, "mean_cond_kl_loss", 0.5),
#     PercentIncreaseEarlyStopping(rec_col.dev, "mouse_cond_kl_loss", 0.25, 10),
#     PercentIncreaseEarlyStopping(rec_col.dev, "mouse_cond_kl_loss", 0.5),
#     GradientClipping(-1000,1000),
#     RandomNegativeSampling(num_boxes, rns_ratio),
    StopIfNaN(),
)

# l = Learner(train_dl, box_model, loss_func, opt, callbacks, recorder = rec_col.learn)
l = Learner(train_dl, box_model, loss_func, opt, callbacks, recorder = rec_col.learn, categories=True)

In [292]:
nEpochs = 100
l.train(nEpochs)

evaluation_human_metric_hard_accuracy tensor(0.5000000000000000)
evaluation_human_metric_hard_f1 tensor(nan)
evaluation_mouse_metric_hard_accuracy_1 tensor(0.5000000000000000)
evaluation_mouse_metric_hard_f1_1 tensor(nan)


align_evaluation_train_align_0.5_metric_hard_accuracy_align tensor(0.6863149404525757)
align_evaluation_train_align_0.5_metric_hard_f1_align tensor(0.6793088912963867)
align_evaluation_train_align_0.5_metric_hard_accuracy_align_mean tensor(0.6434460282325745)
align_evaluation_train_align_0.5_metric_hard_f1_align_mean tensor(0.7219544053077698)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align tensor(0.4587458670139313)
align_evaluation_dev_align_0.5_metric_hard_f1_align tensor(0.3109243512153625)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align_mean tensor(0.4224422574043274)
align_evaluation_dev_align_0.5_metric_hard_f1_align_mean tensor(0.4696969389915466)
evaluation_human_metric_hard_accuracy tensor(0.7182055711746216)
evaluation_human_metric_hard_f1 tensor(0.6280896663665771)
evaluation_mouse_metric_hard_accuracy_1 tensor(0.7900608777999878)
evaluation_mouse_metric_hard_f1_1 tensor(0.7530636787414551)


align_evaluation_train_align_0.5_metric_hard_accuracy_align tensor(0.6995053291320801)
align_evaluation_train_align_0.5_metric_hard_f1_align tensor(0.7384284138679504)
align_evaluation_train_align_0.5_metric_hard_accuracy_align_mean tensor(0.5869744420051575)
align_evaluation_train_align_0.5_metric_hard_f1_align_mean tensor(0.7075306177139282)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align tensor(0.4884488582611084)
align_evaluation_dev_align_0.5_metric_hard_f1_align tensor(0.4599303007125854)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align_mean tensor(0.4372937381267548)
align_evaluation_dev_align_0.5_metric_hard_f1_align_mean tensor(0.5633803009986877)
evaluation_human_metric_hard_accuracy tensor(0.8630952239036560)
evaluation_human_metric_hard_f1 tensor(0.8665629029273987)
evaluation_mouse_metric_hard_accuracy_1 tensor(0.8766365647315979)
evaluation_mouse_metric_hard_f1_1 tensor(0.8825697898864746)


align_evaluation_train_align_0.5_metric_hard_accuracy_align tensor(0.6953833699226379)
align_evaluation_train_align_0.5_metric_hard_f1_align tensor(0.7354099154472351)
align_evaluation_train_align_0.5_metric_hard_accuracy_align_mean tensor(0.5795547962188721)
align_evaluation_train_align_0.5_metric_hard_f1_align_mean tensor(0.7038327455520630)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align tensor(0.5016501545906067)
align_evaluation_dev_align_0.5_metric_hard_f1_align tensor(0.5160256028175354)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align_mean tensor(0.4488448798656464)
align_evaluation_dev_align_0.5_metric_hard_f1_align_mean tensor(0.5824999809265137)
evaluation_human_metric_hard_accuracy tensor(0.8699187040328979)
evaluation_human_metric_hard_f1 tensor(0.8813559412956238)
evaluation_mouse_metric_hard_accuracy_1 tensor(0.8743315339088440)
evaluation_mouse_metric_hard_f1_1 tensor(0.8860462903976440)


align_evaluation_train_align_0.5_metric_hard_accuracy_align tensor(0.7102226018905640)
align_evaluation_train_align_0.5_metric_hard_f1_align tensor(0.7563257813453674)
align_evaluation_train_align_0.5_metric_hard_accuracy_align_mean tensor(0.5816158056259155)
align_evaluation_train_align_0.5_metric_hard_f1_align_mean tensor(0.7050275802612305)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align tensor(0.5247524976730347)
align_evaluation_dev_align_0.5_metric_hard_f1_align tensor(0.5384615659713745)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align_mean tensor(0.4636963605880737)
align_evaluation_dev_align_0.5_metric_hard_f1_align_mean tensor(0.5992602109909058)
evaluation_human_metric_hard_accuracy tensor(0.8699187040328979)
evaluation_human_metric_hard_f1 tensor(0.8835759162902832)
evaluation_mouse_metric_hard_accuracy_1 tensor(0.8708279728889465)
evaluation_mouse_metric_hard_f1_1 tensor(0.8845678567886353)


...

align_evaluation_train_align_0.5_metric_hard_accuracy_align tensor(0.8874691128730774)
align_evaluation_train_align_0.5_metric_hard_f1_align tensor(0.8988513946533203)
align_evaluation_train_align_0.5_metric_hard_accuracy_align_mean tensor(0.8103874921798706)
align_evaluation_train_align_0.5_metric_hard_f1_align_mean tensor(0.8406098484992981)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align tensor(0.6386138796806335)
align_evaluation_dev_align_0.5_metric_hard_f1_align tensor(0.6068222522735596)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align_mean tensor(0.6765676736831665)
align_evaluation_dev_align_0.5_metric_hard_f1_align_mean tensor(0.7003058195114136)
evaluation_human_metric_hard_accuracy tensor(0.8691927790641785)
evaluation_human_metric_hard_f1 tensor(0.8843240141868591)
evaluation_mouse_metric_hard_accuracy_1 tensor(0.8740549683570862)
evaluation_mouse_metric_hard_f1_1 tensor(0.8881244659423828)


align_evaluation_train_align_0.5_metric_hard_accuracy_align tensor(0.8870568871498108)
align_evaluation_train_align_0.5_metric_hard_f1_align tensor(0.8985185027122498)
align_evaluation_train_align_0.5_metric_hard_accuracy_align_mean tensor(0.8103874921798706)
align_evaluation_train_align_0.5_metric_hard_f1_align_mean tensor(0.8406098484992981)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align tensor(0.6386138796806335)
align_evaluation_dev_align_0.5_metric_hard_f1_align tensor(0.6068222522735596)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align_mean tensor(0.6749175190925598)
align_evaluation_dev_align_0.5_metric_hard_f1_align_mean tensor(0.6992366909980774)
evaluation_human_metric_hard_accuracy tensor(0.8691927790641785)
evaluation_human_metric_hard_f1 tensor(0.8843240141868591)
evaluation_mouse_metric_hard_accuracy_1 tensor(0.8740549683570862)
evaluation_mouse_metric_hard_f1_1 tensor(0.8881244659423828)


align_evaluation_train_align_0.5_metric_hard_accuracy_align tensor(0.8874691128730774)
align_evaluation_train_align_0.5_metric_hard_f1_align tensor(0.8988513946533203)
align_evaluation_train_align_0.5_metric_hard_accuracy_align_mean tensor(0.8103874921798706)
align_evaluation_train_align_0.5_metric_hard_f1_align_mean tensor(0.8406098484992981)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align tensor(0.6386138796806335)
align_evaluation_dev_align_0.5_metric_hard_f1_align tensor(0.6068222522735596)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align_mean tensor(0.6749175190925598)
align_evaluation_dev_align_0.5_metric_hard_f1_align_mean tensor(0.6992366909980774)
evaluation_human_metric_hard_accuracy tensor(0.8691927790641785)
evaluation_human_metric_hard_f1 tensor(0.8843240141868591)
evaluation_mouse_metric_hard_accuracy_1 tensor(0.8740549683570862)
evaluation_mouse_metric_hard_f1_1 tensor(0.8881244659423828)


align_evaluation_train_align_0.5_metric_hard_accuracy_align tensor(0.8874691128730774)
align_evaluation_train_align_0.5_metric_hard_f1_align tensor(0.8988513946533203)
align_evaluation_train_align_0.5_metric_hard_accuracy_align_mean tensor(0.8103874921798706)
align_evaluation_train_align_0.5_metric_hard_f1_align_mean tensor(0.8406098484992981)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align tensor(0.6386138796806335)
align_evaluation_dev_align_0.5_metric_hard_f1_align tensor(0.6068222522735596)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align_mean tensor(0.6749175190925598)
align_evaluation_dev_align_0.5_metric_hard_f1_align_mean tensor(0.6992366909980774)
evaluation_human_metric_hard_accuracy tensor(0.8691927790641785)
evaluation_human_metric_hard_f1 tensor(0.8843240141868591)
evaluation_mouse_metric_hard_accuracy_1 tensor(0.8739627599716187)
evaluation_mouse_metric_hard_f1_1 tensor(0.8880518078804016)


align_evaluation_train_align_0.5_metric_hard_accuracy_align tensor(0.8874691128730774)
align_evaluation_train_align_0.5_metric_hard_f1_align tensor(0.8988513946533203)
align_evaluation_train_align_0.5_metric_hard_accuracy_align_mean tensor(0.8103874921798706)
align_evaluation_train_align_0.5_metric_hard_f1_align_mean tensor(0.8406098484992981)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align tensor(0.6386138796806335)
align_evaluation_dev_align_0.5_metric_hard_f1_align tensor(0.6068222522735596)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align_mean tensor(0.6749175190925598)
align_evaluation_dev_align_0.5_metric_hard_f1_align_mean tensor(0.6992366909980774)
evaluation_human_metric_hard_accuracy tensor(0.8691927790641785)
evaluation_human_metric_hard_f1 tensor(0.8843240141868591)
evaluation_mouse_metric_hard_accuracy_1 tensor(0.8739627599716187)
evaluation_mouse_metric_hard_f1_1 tensor(0.8880518078804016)



In [232]:
nEpochs = 100
l.train(nEpochs)

evaluation_human_metric_hard_accuracy tensor(0.5000000000000000)
evaluation_human_metric_hard_f1 tensor(nan)
evaluation_mouse_metric_hard_accuracy_1 tensor(0.5000000000000000)
evaluation_mouse_metric_hard_f1_1 tensor(nan)


align_evaluation_train_align_0.5_metric_hard_accuracy_align tensor(0.5032976269721985)
align_evaluation_train_align_0.5_metric_hard_f1_align tensor(0.0131040131673217)
align_evaluation_train_align_0.5_metric_hard_accuracy_align_mean tensor(0.5173124670982361)
align_evaluation_train_align_0.5_metric_hard_f1_align_mean tensor(0.0684168711304665)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align tensor(0.5000000000000000)
align_evaluation_dev_align_0.5_metric_hard_f1_align tensor(nan)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align_mean tensor(0.5000000000000000)
align_evaluation_dev_align_0.5_metric_hard_f1_align_mean tensor(0.0065573775209486)
evaluation_human_metric_hard_accuracy tensor(0.7224158048629761)
evaluation_human_metric_hard_f1 tensor(0.6358095407485962)
evaluation_mouse_metric_hard_accuracy_1 tensor(0.7901530265808105)
evaluation_mouse_metric_hard_f1_1 tensor(0.7525548934936523)


align_evaluation_train_align_0.5_metric_hard_accuracy_align tensor(0.5251442790031433)
align_evaluation_train_align_0.5_metric_hard_f1_align tensor(0.1111111044883728)
align_evaluation_train_align_0.5_metric_hard_accuracy_align_mean tensor(0.6739488840103149)
align_evaluation_train_align_0.5_metric_hard_f1_align_mean tensor(0.5598219633102417)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align tensor(0.5016501545906067)
align_evaluation_dev_align_0.5_metric_hard_f1_align tensor(0.0503144674003124)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align_mean tensor(0.5099009871482849)
align_evaluation_dev_align_0.5_metric_hard_f1_align_mean tensor(0.1633802801370621)
evaluation_human_metric_hard_accuracy tensor(0.8712252974510193)
evaluation_human_metric_hard_f1 tensor(0.8739877939224243)
evaluation_mouse_metric_hard_accuracy_1 tensor(0.8803245425224304)
evaluation_mouse_metric_hard_f1_1 tensor(0.8865186572074890)


align_evaluation_train_align_0.5_metric_hard_accuracy_align tensor(0.5399835109710693)
align_evaluation_train_align_0.5_metric_hard_f1_align tensor(0.1889534890651703)
align_evaluation_train_align_0.5_metric_hard_accuracy_align_mean tensor(0.7757625579833984)
align_evaluation_train_align_0.5_metric_hard_f1_align_mean tensor(0.7536231875419617)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align tensor(0.5132012963294983)
align_evaluation_dev_align_0.5_metric_hard_f1_align tensor(0.1399416923522949)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align_mean tensor(0.5198019742965698)
align_evaluation_dev_align_0.5_metric_hard_f1_align_mean tensor(0.2595419585704803)
evaluation_human_metric_hard_accuracy tensor(0.8734030127525330)
evaluation_human_metric_hard_f1 tensor(0.8837953209877014)
evaluation_mouse_metric_hard_accuracy_1 tensor(0.8755301237106323)
evaluation_mouse_metric_hard_f1_1 tensor(0.8868967890739441)


align_evaluation_train_align_0.5_metric_hard_accuracy_align tensor(0.5560593605041504)
align_evaluation_train_align_0.5_metric_hard_f1_align tensor(0.2628336548805237)
align_evaluation_train_align_0.5_metric_hard_accuracy_align_mean tensor(0.8050288558006287)
align_evaluation_train_align_0.5_metric_hard_f1_align_mean tensor(0.8010096549987793)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align tensor(0.5297029614448547)
align_evaluation_dev_align_0.5_metric_hard_f1_align tensor(0.1971831023693085)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align_mean tensor(0.5181518197059631)
align_evaluation_dev_align_0.5_metric_hard_f1_align_mean tensor(0.3080568611621857)
evaluation_human_metric_hard_accuracy tensor(0.8735482096672058)
evaluation_human_metric_hard_f1 tensor(0.8861289024353027)
evaluation_mouse_metric_hard_accuracy_1 tensor(0.8717499375343323)
evaluation_mouse_metric_hard_f1_1 tensor(0.8852593302726746)


...

align_evaluation_train_align_0.5_metric_hard_accuracy_align tensor(0.8878812789916992)
align_evaluation_train_align_0.5_metric_hard_f1_align tensor(0.8891605734825134)
align_evaluation_train_align_0.5_metric_hard_accuracy_align_mean tensor(0.8441879749298096)
align_evaluation_train_align_0.5_metric_hard_f1_align_mean tensor(0.8651925921440125)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align tensor(0.5379537940025330)
align_evaluation_dev_align_0.5_metric_hard_f1_align tensor(0.2893401086330414)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align_mean tensor(0.5016501545906067)
align_evaluation_dev_align_0.5_metric_hard_f1_align_mean tensor(0.3786008059978485)
evaluation_human_metric_hard_accuracy tensor(0.8709349632263184)
evaluation_human_metric_hard_f1 tensor(0.8856886029243469)
evaluation_mouse_metric_hard_accuracy_1 tensor(0.8736861348152161)
evaluation_mouse_metric_hard_f1_1 tensor(0.8878520131111145)


align_evaluation_train_align_0.5_metric_hard_accuracy_align tensor(0.8973619341850281)
align_evaluation_train_align_0.5_metric_hard_f1_align tensor(0.8994751572608948)
align_evaluation_train_align_0.5_metric_hard_accuracy_align_mean tensor(0.8450123667716980)
align_evaluation_train_align_0.5_metric_hard_f1_align_mean tensor(0.8658101558685303)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align tensor(0.5313531160354614)
align_evaluation_dev_align_0.5_metric_hard_f1_align tensor(0.2791878283023834)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align_mean tensor(0.5000000000000000)
align_evaluation_dev_align_0.5_metric_hard_f1_align_mean tensor(0.3778234124183655)
evaluation_human_metric_hard_accuracy tensor(0.8687572479248047)
evaluation_human_metric_hard_f1 tensor(0.8839836120605469)
evaluation_mouse_metric_hard_accuracy_1 tensor(0.8747925758361816)
evaluation_mouse_metric_hard_f1_1 tensor(0.8887249827384949)


align_evaluation_train_align_0.5_metric_hard_accuracy_align tensor(0.9097279310226440)
align_evaluation_train_align_0.5_metric_hard_f1_align tensor(0.9132672548294067)
align_evaluation_train_align_0.5_metric_hard_accuracy_align_mean tensor(0.8441879749298096)
align_evaluation_train_align_0.5_metric_hard_f1_align_mean tensor(0.8651925921440125)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align tensor(0.5313531160354614)
align_evaluation_dev_align_0.5_metric_hard_f1_align tensor(0.2864321768283844)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align_mean tensor(0.4966996610164642)
align_evaluation_dev_align_0.5_metric_hard_f1_align_mean tensor(0.3762781023979187)
evaluation_human_metric_hard_accuracy tensor(0.8694831728935242)
evaluation_human_metric_hard_f1 tensor(0.8845511674880981)
evaluation_mouse_metric_hard_accuracy_1 tensor(0.8739627599716187)
evaluation_mouse_metric_hard_f1_1 tensor(0.8880700469017029)


align_evaluation_train_align_0.5_metric_hard_accuracy_align tensor(0.9163231849670410)
align_evaluation_train_align_0.5_metric_hard_f1_align tensor(0.9196676015853882)
align_evaluation_train_align_0.5_metric_hard_accuracy_align_mean tensor(0.8421269655227661)
align_evaluation_train_align_0.5_metric_hard_f1_align_mean tensor(0.8636525869369507)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align tensor(0.5330032706260681)
align_evaluation_dev_align_0.5_metric_hard_f1_align tensor(0.2871536612510681)
align_evaluation_dev_align_0.5_metric_hard_accuracy_align_mean tensor(0.4966996610164642)
align_evaluation_dev_align_0.5_metric_hard_f1_align_mean tensor(0.3762781023979187)
evaluation_human_metric_hard_accuracy tensor(0.8700639009475708)
evaluation_human_metric_hard_f1 tensor(0.8850058317184448)
evaluation_mouse_metric_hard_accuracy_1 tensor(0.8753457665443420)
evaluation_mouse_metric_hard_f1_1 tensor(0.8891621828079224)



Stopped training at 24.548022598870055 epochs due to keyboard interrupt.
